# Coupled-perturbed SCF

Coupled-perturbed SCF (CP-SCF) is a method to compute __static__ (i.e. time-independent) __molecular properties__ from a reference wavefunction. In this Jupyter notebook, we will use a Hartree-Fock wavefunction (dubbed CP-HF) to compute the static, electric dipole polarizability tensor for the water molecule.

## Basics of Polarization

In some cases, take dielectrics for example, we would like to know how these materials behave under the influence of an external electric field. Here, we attempt to compute the __electric dipole polarizability__ $\alpha_{ij}$ with Hartree-Fock theory.

How do we connect Hartree-Fock to the polarizability? Any Hartree-Fock calculation results in an energy, $E_\text{HF}$, so we need a relation between the energy and the polarizability. A direct connection between these two quantities is not immediately obvious, but we know how to (i) compute the energy of a dipole moment in an electric field and (ii) how the polarizability is related to the dipole moment.

From basic electrostatics, we know how to compute the energy of a dipole in a homogeneous electric field: It is the (negative of the) dot product between the __dipole moment__ $\vec{\mu}$ and the electric field $\vec{E}$. Here, the index $i$ denotes one of the directions of space, e.g. $x$.

\begin{align}
    E &= - \vec{\mu} \cdot \vec{E}\\
    \Leftrightarrow \quad \mu_i &= - \frac{\partial E}{\partial E_i}
\end{align}

The total dipole moment can be written as a sum of a permanent contribution (think of the water molecule here) and a part that is __induced__ by an external field. The dipole polarizability $\alpha$ describes how much the total dipole moment reacts to the external field:

\begin{align}
    \mu_{i} &= \mu_{i,\text{permanent}} + \sum_j \left.\frac{\partial \mu_i}{\partial E_j}\right|_{\vec{E}=0} E_j + \text{higher orders}\\
            &= \mu_{i,\text{permanent}} + \sum_j \alpha_{ij} E_j + \text{higher orders}
\end{align}

Again, the indices $i$ and $j$ refer to the three spatial dimensions $x$, $y$, and $z$. From the two equations, we identify the polarizability to be

\begin{align}
    \alpha_{ij} &= \left.\frac{\partial \mu_i}{\partial E_j}\right|_{\vec{E}=0}
\end{align}

Combined with the previous definition of $ \mu_{i} $, we can write

\begin{align}
    \alpha_{ij} &= - \frac{\partial^2 E}{\partial E_i \partial E_j}
\end{align}

We can compute the polarizability by differentiating the Hartree-Fock energy twice with respect to the fields in $i$ and $j$ direction. Now, we only have to think about how to actually differentiate the Hartree-Fock energy.

## Linear Response Theory

Linear Response Theory is a useful method to compute _ab initio_ molecular properties. Here, we focus on the time-independent response of a molecular system. From the above, we know that we need to compute the second derivative of the energy with respect to electric fields, and linear response theory can tell us how to do that. This section is kept as general as possible, and the next section will give the explicit expressions used later in the computational procedure.

First of all, we have to define our wavefunction and the Hamiltonian. The wavefunction $\psi$ is parameterized as a function of the __wavefunction parameter__ $\lambda$ as $\psi(\lambda)$ (allows for easy mathematical manipulation) and our Hamiltonian is written as $\hat{H}(x) = \hat{H}_0 + V(x)$ with $V(x)$ being the __perturbation__ of the system. The $x$ denotes the __perturbation parameter__, which is the electric field in our case. Keep in mind that the wavefunction parameter depends on the perturbation parameter as $\lambda(x)$! This is because the wavefunction will be influenced by an external perturbation.

For the energy of the system under a perturbation of $V(x)$ we can now write

\begin{align}
    E (x, \lambda) &= \langle \psi(\lambda) | \hat{H}(x) | \psi(\lambda) \rangle
\end{align}

Remember, we need derivatives of the energy to compute our desired property. So let's differentiate the energy with respect to the perturbation parameter:

\begin{align}
    \frac{\mathrm{d} E(x, \lambda_0)}{\mathrm{d} x} &= \underbrace{\frac{\partial E(x, \lambda_0)}{\partial x}}_\text{explicit dependence} + \underbrace{\left.\frac{\partial E(x, \lambda)}{\partial \lambda}\right|_{\lambda=\lambda_0} \left.\frac{\partial \lambda}{\partial x}\right|_{\lambda=\lambda_0}}_\text{implicit dependence}
\end{align}

where we have used the chain rule for the implicit-dependence part. The subscripts $\lambda=\lambda_0$ indicate that the derivatives are evaluated at the parameters of a converged wavefunction, $\lambda_0$.

The expression for the first derivative can be simplified: If $\partial E / \partial \lambda$ is evaluated at the wavefunction parameter of an optimized wavefunction, we know that this term will be exactly zero since we are in a minimum! We arrive at

\begin{align}
    \frac{\mathrm{d} E(x, \lambda_0)}{\mathrm{d} x} &= \frac{\partial E(x, \lambda_0)}{\partial x}
\end{align}

Now we can tackle the second derivatives. We differentiate the above expression again to get

\begin{align}
    \frac{\mathrm{d}^2 E(x, \lambda_0)}{\mathrm{d} x^2} &= \frac{\partial^2 E(x, \lambda_0)}{\partial x^2} + \left.\frac{\partial E(x, \lambda)}{\partial x \partial \lambda}\right|_{\lambda=\lambda_0} \left.\frac{\partial \lambda}{\partial x}\right|_{\lambda=\lambda_0}
\end{align}

So far, so good. However, we now need an expression for $\partial \lambda / \partial x$, the __wavefunction response__ (turns out, we know how to compute the rest, see next section). We can get it with a trick: Remember that _for any perturbation parameter_ $x$

\begin{align}
    \left.\frac{\partial E(x, \lambda)}{\partial \lambda}\right|_{\lambda=\lambda_0} = 0 \quad \forall x
\end{align}

We can differentiate this condition with respect to $x$ to obtain our desired wavefunction response!

\begin{align}
    \frac{\mathrm{d}}{\mathrm{d}x} \left.\frac{\partial E(x, \lambda)}{\partial \lambda}\right|_{\lambda=\lambda_0} &= \left.\frac{\partial^2 E(x,\lambda)}{\partial x \partial \lambda}\right|_{\lambda=\lambda_0} + \left.\frac{\partial^2 E(x,\lambda)}{\partial \lambda^2}\right|_{\lambda=\lambda_0} \left.\frac{\partial \lambda}{\partial x}\right|_{\lambda=\lambda_0} = 0\\
    \Leftrightarrow \quad \underbrace{\left.\frac{\partial^2 E(x,\lambda)}{\partial \lambda^2}\right|_{\lambda=\lambda_0}}_\text{electronic 
    Hessian} \left.\frac{\partial \lambda}{\partial x}\right|_{\lambda=\lambda_0} &= - \underbrace{\left.\frac{\partial^2 E(x,\lambda)}{\partial x \partial \lambda}\right|_{\lambda=\lambda_0}}_{\substack{\text{perturbation tensor} \\ \text{"right-hand side"}}}
\end{align}

_Interesting deviation_: Doesn't the last equation remind you of something? Think of __Hooke's law__ for springs:

\begin{align}
    k x &= -F
\end{align}

Hooke's law tells us how a mass on a spring reacts to an applied force. In our case, the equation describes how the wavefunction will react to an applied perturbation. We can view this as the orbitals changing slightly (remember, a perturbation has to be small for perturbation theory to be applicable!) to new orbitals that are optimal under the perturbation, called __orbital rotation__. You can also think about it as the electrons being __excited__ from the occupied to the unoccupied orbitals in response to the perturbation (the mathematical connection will be made clear in the next section).

## Coupled-Perturbed Hartree-Fock

We are finally ready to apply all of the above to our problem! To summarize the above, we need the following equations (of course, we need to replace the generic perturbation parameter $x$ with the electric field components $E_i$ and $E_j$):

\begin{align}
    \alpha_{ij} &= - \frac{\partial^2 E}{\partial E_i \partial E_j}\\
    &= - \left[\frac{\partial^2 E(\vec{E}, \lambda_0)}{\partial E_i \partial E_j} + \left.\frac{\partial E(\vec{E}, \lambda)}{\partial E_i \partial \lambda}\right|_{\lambda=\lambda_0} \left.\frac{\partial \lambda}{\partial E_j}\right|_{\lambda=\lambda_0}\right]\\
    \left.\frac{\partial^2 E(\vec{E},\lambda)}{\partial \lambda^2}\right|_{\lambda=\lambda_0} \left.\frac{\partial \lambda}{\partial E_i}\right|_{\lambda=\lambda_0} &= - \left.\frac{\partial^2 E(\vec{E},\lambda)}{\partial E_i \partial \lambda}\right|_{\lambda=\lambda_0}
\end{align}

Now we have to find explicit expression for Hartree-Fock theory and the polarizability. Let's start again with the parameterization of the wavefunction: This can be conveniently done in Hartree-Fock theory with an __orbital rotation operator__ $\hat{\kappa}$ (see Helgaker, chapters 3 and 10.1.2)

\begin{align}
    \hat{\kappa} &= \sum_{p>q} \kappa_{pq} E^{-}_{pq}\\
    E^{-}_{pq} &= E_{pq} - E_{qp}\\
    &= (a_{p\alpha}^\dagger a_{q\alpha} + a_{p\beta}^\dagger a_{q\beta}) - (a_{q\alpha}^\dagger a_{p\alpha} + a_{q\beta}^\dagger a_{p\beta})
\end{align}

$E_{pq}$ is a __singlet excitation operator__; it raises a pair ($\alpha$ and $\beta$) of electrons from orbital $p$ to $q$ (remember the last paragraph in the last section!). Now, the wavefunction can be written like

\begin{align}
    \psi (\kappa) &= \exp(-\hat{\kappa}) \psi
\end{align}

Here, $\exp(-\hat{\kappa})$ is a _unitary operator_ because the orbital rotation operator is _anti-Hermitian_: Every unitary matrix may be written as an exponential of an anti-Hermitian matrix. This is where the term __orbital rotation__ can easily be rationalized: If you apply a unitary matrix on a vector, the vector's length remains unchanged but it 'rotates' in space.

Now for the Hamiltonian: In our case, the perturbation is

\begin{align}
    V(\vec{E}) &= -\hat{\mu} \vec{E}\\
    \hat{H}(\vec{E}) &= \hat{H}_0 + V(\vec{E})
\end{align}

With this, we can already simplify the response equation from above: Differentiating $\partial^2 E(\vec{E}, \kappa_0) / \partial E_i \partial E_j$ twice is zero here because $(\partial^2 / \partial E_i \partial E_j) \hat{\mu} \vec{E} = 0$.

Now we only need two more explicit expressions: One for the electronic Hessian to solve for the wavefunction response and our perturbation vectors. Thankfully, these are known in the literature ([Jørgensen 1975](https://dx.doi.org/10.1146/annurev.pc.26.100175.002043), p. 368). The electronic Hessian for our closed-shell HF calculation reads

\begin{align}
    \left.\frac{\partial^2 E(x,\kappa)}{\partial \kappa^2}\right|_{\kappa=0} &= \mathbf{A} + \mathbf{B}\\
    A_{iajb} &= \delta_{ij} F_{ab} - \delta_{ab} F_{ij} + 2 g_{aijb} - g_{jiab}\\
    B_{iajb} &= 2 g_{aijb} - g_{ajbi}
\end{align}

Here, $\mathbf{F}$ is the molecular Fock matrix and $g_{pqrs}$ denotes the two-electron repulsion integrals. The molecular Hessian was split into two matrices $\mathbf{A}$ and $\mathbf{B}$ (which occur explicitly in TDSCF calculations).

The perturbation vectors ("right-hand sides"), from now on denoted with $\mathbf{f}$, are

\begin{align}
    (\mathbf{f}_i)_{rs} &= \left.\frac{\partial E(\vec{E}, \kappa)}{\partial \vec{E}_i \partial \kappa_{rs}}\right|_{\kappa=0} = -2 \langle r | \hat{\mu}_i | s \rangle
\end{align}

where $\hat{\mu}_i$ is the __dipole moment operator__.

With this knowledge, we can assemble the final equations. We also introduce the abbreviation for the wavefunction response vector $\mathbf{x}_j = \partial \kappa / \partial E_j$.

\begin{align}
    \alpha_{ij} &= - \left[\underbrace{\frac{\partial^2 E(\vec{E}, \kappa_0)}{\partial E_i \partial E_j}}_{=0} + \left.\frac{\partial E(\vec{E}, \kappa)}{\partial E_i \partial \kappa}\right|_{\kappa=0} \left.\frac{\partial \kappa}{\partial E_j}\right|_{\kappa=0}\right]\\
    &= - \mathbf{f}_i \mathbf{x}_j\\
    (\mathbf{A}+\mathbf{B}) \mathbf{x}_i &= - \mathbf{f}_i
\end{align}

Therefore, the polarizability can be computed as
\begin{align}
    \alpha_{ij} &= \mathbf{f}_i\, (\mathbf{A} - \mathbf{B})^{-1}\, \mathbf{f}_j
\end{align}

## Computational Procedure

For our computation, we need the matrices and vectors that were discussed in the theory section. We will use Psi4 to compute these for a converged Hartree-Fock wavefunction.

We use `numpy` to expose the perturbation tensors $\mathbf{f}$, the occupation matrix $\mathbf{C}$, the Fock matrix $\mathbf{F}$ and the electron repulsion integrals $\mathbf{I}$ to Python such that we can use `numpy`'s matrix/vector manipulations.

In [1]:
import psi4

import numpy as np
np.set_printoptions(3, linewidth=100, suppress=True)    # when we inspect the vectors/matrices,
                                                        # use a prettier format for printing

The geometry of the water molecule is set to a previously optimized one for the aug-cc-pVDZ basis. We use the keyword `symmetry c1` to tell Psi4 not to use the actual point group $C_{2v}$ in the computation: If Psi4 used symmetry, we would not be able to create correct matrices for use in `numpy`.

In [2]:
mol = psi4.geometry('''
    O
    H  1  0.9435
    H  1  0.9435  2  105.9443
    symmetry c1
''')

Now, we set the basis to aug-cc-pVDZ. This is the same basis set that was used for the geometry optimization.

In [3]:
psi4.set_options({'BASIS': 'aug-cc-pVDZ'})

We are now ready to run the SCF cycle to converge the wavefunction. By telling Psi4 to `return_wfn=True`, we get back the converged wavefunction from which we can extract the pertinent matrices. We print out the energy such that we can compare it against a pre-computed reference. It needs not exactly match the reference, slight variations are OK.

In [4]:
scf_energy, wfn = psi4.energy('SCF', return_wfn=True)
print('SCF Energy: {}'.format(scf_energy))  # Print the final, converged SCF energy, should be something like
                                            # -76.04182368701949

SCF Energy: -76.04182368620263


From now on, we need the orbital occupation matrix $\mathbf{C}$: The quantities that we get from Psi4 are in the spin-orbital basis. We, on the other hand, need them to be in the molecular orbital basis. The $\mathbf{C}$ matrix is the key to this as it tells us how to get from one basis to the other. The rows are the spin orbitals $\lambda$, and the coefficients in the columns tell us how to construct the corresponding molecular orbital $p$ from the spin orbitals:

\begin{align}
    \chi_p &= \sum_\lambda \phi_\lambda C_{\lambda p}
\end{align}

Similarly, we can transform any index from the SO to the MO basis.

Note that the code says `wfn.Ca`: We get the coefficients matrix for the $\alpha$ spins. However, since we are working in restricted Hartree-Fock (RHF) where spins are always paired, this does not really matter.

In [5]:
C = np.asarray(wfn.Ca())

We use the transformation with the Fock matrix first. In the first line of the following code, we get the Fock matrix in the SO basis with the elements

\begin{align}
    F_{\lambda \mu}
\end{align}

To transform the first index, we left-multiply with the transpose of the coefficients matrix

\begin{align}
    F_{p\mu} &= \sum_\lambda C_{\lambda p} F_{\lambda \mu}\\
    &= \sum_\lambda C_{p \lambda}^T F_{\lambda\mu}\\
    \mathbf{F}' &= \mathbf{C}^T \mathbf{F}
\end{align}

We can do the same for the second index on the intermediate matrix $\mathbf{F}'$ to arrive at the molecular Fock matrix:

\begin{align}
    F_{pq} &= \sum_{\mu} F_{p\mu} C_{\mu q}\\
    \mathbf{F}_\text{mol} &= \mathbf{F}' \mathbf{C}\\
    &= \mathbf{C}^T \mathbf{F} \mathbf{C}
\end{align}

In [6]:
F = np.asarray(wfn.Fa())
F = np.dot(C.T, F).dot(C)

In the following computations, we will need the electron repulsion integrals. These can be computed with a `psi4.core.MintsHelper`. The `MintsHelper` is told to use the same basis that was used for the wavefunction to compute the basis-dependent integrals.

In [7]:
mints = psi4.core.MintsHelper(wfn.basisset())

The electron repulsion integrals are obtained by `mints.ao_eri()` and exposed to `numpy`. These integrals are obtained in the atomic orbital basis (AO). Since we are in RHF, we can use them with the other quantities that are computed for the SO basis.

In [8]:
I = np.asarray(mints.ao_eri())

These integrals have to be transformed to the MO basis in the same manner as was done for the Fock matrix. Here, we use `numpy`'s Einstein summation function. Albert Einstein introduced in an article on "[The basis of general relativity](https://dx.doi.org/10.1002/andp.19163540702)" (p. 781) in 1916, where he states that "Should an index appear in a term of an expression twice, then it is always summed over, except it is explicitly noted otherwise."

Here, for the transformation of the first index, we get

\begin{align}
    g_{p\mu\nu\sigma} &= \sum_\lambda g_{\lambda\mu\nu\sigma} C_{\lambda p} \equiv g_{\lambda\mu\nu\sigma} C_{\lambda p}
\end{align}

In `numpy.einsum`, these indices would be written as $\lambda\mu\nu\sigma, \lambda p \rightarrow p\mu\nu\sigma$ and so on for the remaining indices.

In [9]:
g_pqrs = np.einsum('pqrs,pt->tqrs', I, C)
g_pqrs = np.einsum('pqrs,qt->ptrs', g_pqrs, C)
g_pqrs = np.einsum('pqrs,rt->pqts', g_pqrs, C)
g_pqrs = np.einsum('pqrs,st->pqrt', g_pqrs, C)

We are now ready to build the orbital rotation matrices $\mathbf{A}$ and $\mathbf{B}$. Recall from above that

\begin{align}
    A_{iajb} &= \delta_{ij} F_{ab} - \delta_{ab} F_{ij} + 2 g_{aijb} - g_{jiab}
\end{align}

The Kronecker deltas are represented by a unit matrix `numpy.eye(...)` which has the same shape as the molecular Fock matrix.

In [10]:
A = np.einsum('ij,ab->iajb', np.eye(F.shape[0]), F)
A -= np.einsum('ab,ij->iajb', np.eye(F.shape[0]), F)
A += 2 * np.einsum('aijb->iajb', g_pqrs)
A -= np.einsum('jiab->iajb', g_pqrs)

The $\mathbf{B}$ matrix is computed from

\begin{align}
    B_{iajb} &= 2 g_{aijb} - g_{ajbi}
\end{align}

and thus only contains contributions from the electron repulsion integrals.

In [11]:
B = 2 * np.einsum('aijb->iajb', g_pqrs)
B -= np.einsum('ajbi->iajb', g_pqrs)

The electronic Hessian is then simply the sum of these two matrices. When searching the literature, keep in mind that some authors defined their $\mathbf{B}$ with the opposite sign. For them, the Hessian will be the difference of both orbital rotation matrices.

In [12]:
H = A + B

Actually, we do not need the full Hessian, only the parts where the inactive and active orbitals contribute are of importance (see also Helgaker, sections 10.1.2 and 10.2.2). Therefore, we will now aim to extract the relevant parts of the Hessian. We already hinted at this by using the indices $i,j$ for inactive and $a,b$ for virtual orbitals in the definition of the Hessian.

For this, we need to know the number of number of basis functions = number of MOs, the number of doubly occupied orbitals (for RHF, identical to the number of electrons with $\alpha$ spin as all are paired) and the number of virtual orbitals. The number of rotable orbitals, i.e. the possible combinations of inactive and active orbitals, can then be computed by simple multiplication.

In [13]:
nbf = wfn.nmo()
ndocc = wfn.nalpha()
nvirt = nbf - ndocc
nrot = ndocc * nvirt

The elements in the Hessian are sorted, the inactive orbitals come first and the virtual ones second. Therefore, the number of doubly occupied orbitals is the boundary between both, so that we can use `ndocc` for slicing. Remember that we need to slice such that we get

\begin{align}
    H_{iajb}
\end{align}

Finally, to bring the Hessian into matrix form for `numpy.linalg.solve`, we reshape it to be of 'number of rotable' $\times$ 'number of rotable' orbitals.

In [14]:
H = H[:ndocc, ndocc:, :ndocc, ndocc:]
H = H.reshape(nrot, nrot)

We now have everything in place except for the perturbations themselves! These dipole tensors are computed by the `MintsHelper` instance and subsequently scaled by $-2$ to account for the doubly occupied orbitals.

In [15]:
tmp_tensors = mints.so_dipole()     # get the tensors in the spin-orbital basis

dipole_tensors = []
for tensor in tmp_tensors:
    tensor.scale(-2)    # use the correct prefactor
    dipole_tensors.append(np.asarray(tensor))   # convert psi4's internal tensors to a numpy vector

These tensors are in the SO basis, so we have to transform them the same way as done above. Also, same as for the Hessian, they still contain redundant rotations. The vectors are sliced such that we only use the relevant parts and raveled for use with `numpy.linalg.solve`.

In [16]:
for i in range(len(dipole_tensors)):
    dipole_tensors[i] = np.dot(C.T, dipole_tensors[i]).dot(C)
    dipole_tensors[i] = dipole_tensors[i][:ndocc, ndocc:].ravel()

We are finally ready to solve the linear response equations! For each perturbation, solve

\begin{align}
    H_{ia,jb} x_{jb} &= -f_{ia}
\end{align}

and store the resultant responses in `responses`.

In [17]:
responses = []
for perturbation in dipole_tensors:
    responses.append(np.linalg.solve(H, -perturbation))

The static polarizability tensor can be computed by dotting the perturbation vectors with the reponse vectors.

\begin{align}
    \alpha_{ij} &= - \mathbf{f}_i \mathbf{x}_j
\end{align}

This is exactly how the polarizability tensor is usually defined: It tells us how the system responds in $i$-direction to a perturbation along $j$.

In [18]:
polarizabilities = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        polarizabilities[i, j] = -np.dot(dipole_tensors[i], responses[j])

The tensor is:

In [19]:
print(polarizabilities)

[[ 7.259  0.    -0.   ]
 [ 0.     8.797 -0.   ]
 [-0.    -0.     7.854]]


However, the tensor is not invariant to the choice of coordinate system. Usually we are more interested in the three principal components, which are the eigenvalues $[\alpha_1, \alpha_2, \alpha_3]$:

In [20]:
principal_components = np.linalg.eigvals(polarizabilities)
print(principal_components)

[ 7.259  7.854  8.797]


and the isotropic polarizability, which can be defined as

\begin{equation}
\alpha_{\text{iso}} \equiv \frac{1}{3}\mathrm{Tr}(\mathbf{\alpha}) = \frac{1}{3}(\alpha_1 + \alpha_2 + \alpha_3)
\end{equation}

In [21]:
isotropic_polarizability = np.mean(principal_components)
print(np.around(isotropic_polarizability, 4))

7.9699


## References

Trygve Helgaker, Poul Jørgensen, Jeppe Olsen, _Molecular Electronic-Structure Theory_ (John Wiley & Sons, Chichester, 2000). Available in digital form at [doi:10.1002/9781119019572](https://dx.doi.org/10.1002/9781119019572).

Julien Toulouse, _Introduction to the calculation of molecular properties by response theory_, lecture notes for a class given at the Université Pierre et Marie Curie, 2017. Last accessed on March 11, 2017 at [http://www.lct.jussieu.fr/pagesperso/toulouse/enseignement/molecular_properties.pdf](http://www.lct.jussieu.fr/pagesperso/toulouse/enseignement/molecular_properties.pdf).

Trygve Helgaker, _Time-independent molecular properties_, lecture at the 13th Sostrup Summer School Quantum Chemistry and Molecular Properties, 2014. Last accessed on March 11, 2017 at [http://folk.uio.no/helgaker/talks/SostrupTI_14.pdf](http://folk.uio.no/helgaker/talks/SostrupTI_14.pdf).

Poul Jørgensen, _Molecular and Atomic Applications of Time-Dependent Hartree-Fock Theory_, Annual Review of Physical Chemistry __26__, 359-380 (1975). Available in digital form at [doi:10.1146/annurev.pc.26.100175.002043](https://dx.doi.org/10.1146/annurev.pc.26.100175.002043).